# Step 1: Data Preparation: Data Sourcing and Cleaning

In [2]:
# Data sourcing and cleaning
import pandas as pd
import numpy as np
from tensorboard.notebook import display

# Reading the data set 
df = pd.read_csv('../data/raw_data/Children-and-young-ppl-asthma-organisational-audit-2019-20-Data.csv')
print("Original Data")
print(df.head(10))
print("\n \n")
print(df.tail(10))
#Returning how many rows in the df
print("\n The amount of organisations in this data set is",df.count().values[0],".") 


Original Data
  orgcode                          description  \
0     ADD               Addenbrooke's Hospital   
1     AEI        Royal Albert Edward Infirmary   
2     AIR            Airedale General Hospital   
3     ALC  Royal Alexandra Children's Hospital   
4     BAR   Barnsley District General Hospital   
5     BAS                    Basildon Hospital   
6     BAT           Royal United Hospital Bath   
7     BCH       Birmingham Children's Hospital   
8     BED                     Bedford Hospital   
9     BFH                Broomfield Chelmsford   

                                               trust  country emergency_adm  \
0  Cambridge University Hospitals NHS Foundation ...  England             -   
1  Wrightington, Wigan and Leigh NHS Foundation T...  England           400   
2                      Airedale NHS Foundation Trust  England           110   
3  Brighton and Sussex University Hospitals NHS T...  England            10   
4             Barnsley Hospital NHS Foun

Assessing the quality - high quality data set and how does it reflect real life data. It is reliable as it is an open data set which is provided by a creditable source. This is an external data source provided by the data.go.uk of the information of asthma in children and young adults from 2018 - 2019. It is a structured data as it carried out the following characteristics : well organised, table form, and the tables can be related in a database schema. 

From my inital inspection I have noticed that there are a few rows which have empty values. Majority of the rows which have empty values mostly have value which are empty in the first couple of rows (A-G) which is an interger of the type of admission. I believe that it would be benefical to drop these values because the admission data is one of the important columns in the data set, and as these reigons of hospitals differ i would not think i would be benefical to make adverages of these admissions. 

Via inspection I have noticed that some values which are - are also equlient to Nan values, but since python is dropna() function only detects nan values I would have to convert the NaN or Null values 

In [46]:
# Data Cleaning - missing values

# Dropping unuseful columns 


# Handling the different types of na values "-, n/a , null"
print('Replacing the - values with na so it can be detected by dropna \n')
df = df.replace('-',np.nan)

# How many NA values are in a row
print("EMPTY ROWS VALUES")
print(df.head(50).isna().sum(axis=1))
print(df.tail(50).isna().sum(axis=1))

print("EMPTY COLUMNS VALUES")
# How many NA values are in a column
print(df.isna().sum(axis=0))

# Checking if there are missing values in a row
print('Checking for each row is there na values \n ')

Replacing the - values with na so it can be detected by dropna 

EMPTY ROWS VALUES
0     57
1      8
2     12
3      2
4      3
5      9
6     10
7      3
8      5
9      8
10    43
11     6
12     8
13     6
14     5
15    24
16     4
17    40
18     5
19    10
20     7
21    10
22    36
23    60
24     4
25     5
26     3
27     4
28     8
29    11
30     4
31     8
32     4
33     7
34     2
35     3
36    23
37    11
38    62
39     3
40     6
41     2
42    10
43     4
44     3
45    68
46    37
47    36
48     3
49    37
dtype: int64
92     65
93      9
94      1
95      6
96      8
97      1
98     73
99      1
100     8
101     8
102    65
103     5
104     8
105    10
106     6
107    39
108     3
109     8
110     7
111     4
112     2
113     6
114     5
115     7
116     3
117     4
118    35
119     2
120    44
121    65
122    63
123    11
124     6
125     8
126     8
127     4
128    37
129    16
130     3
131     2
132     5
133     3
134    62
135    19
136     5
137 

The statement which I have chosen takes into consideration the amount of na values which is in each row in the different format. Based on the inspection there are many na values in the format - or na, so I have replaced the - values with na so it can be detected. I then checked how many na values are in each row and how many na values are in each column

So it would be appropriate to drop a row or column I would need to have a threshold, 143 by 156 respectively. I believe that if a row or column has too many missing values it would be acceptable to drop them as it wouldn't be useful 

In [ ]:
# duplicate values

In [ ]:
#formatting

In [ ]:
#headers

In [ ]:
# Data transformation - normalise, Discretize or aggregate , new attributes 


In [ ]:
# Data reduction

In [ ]:
# Saving the cleaned data set into a new file 
df.to_csv("cleaned_dataset.csv", index=False)
